In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold,cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor
from sklearn import preprocessing
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score

In [ ]:
company=pd.read_csv('Company.csv')
company

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,Good,33,14,Yes,Yes
396,6.14,139,23,3,37,120,Medium,55,11,No,Yes
397,7.41,162,26,12,368,159,Medium,40,18,Yes,Yes
398,5.94,100,79,7,284,95,Bad,50,12,Yes,Yes


In [ ]:
company.shape

(400, 11)

In [ ]:
company.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


In [ ]:
company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


In [ ]:
company.isnull().sum()

Sales          0
CompPrice      0
Income         0
Advertising    0
Population     0
Price          0
ShelveLoc      0
Age            0
Education      0
Urban          0
US             0
dtype: int64

In [ ]:
le=preprocessing.LabelEncoder()
company['Urban']=le.fit_transform(company['Urban'])
company['US']=le.fit_transform(company['US'])
company['ShelveLoc']=le.fit_transform(company['ShelveLoc'])

In [ ]:
company

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,0,42,17,1,1
1,11.22,111,48,16,260,83,1,65,10,1,1
2,10.06,113,35,10,269,80,2,59,12,1,1
3,7.40,117,100,4,466,97,2,55,14,1,1
4,4.15,141,64,3,340,128,0,38,13,1,0
...,...,...,...,...,...,...,...,...,...,...,...
395,12.57,138,108,17,203,128,1,33,14,1,1
396,6.14,139,23,3,37,120,2,55,11,0,1
397,7.41,162,26,12,368,159,2,40,18,1,1
398,5.94,100,79,7,284,95,0,50,12,1,1


In [ ]:
#encode labels as 0=labels 0=Bad , 1=Good and  2=Medium for shelveloc
#encode  labels as  0 = No and 1 = Yes for urban
#encode labels as  0 = No and 1 = Yes for us

In [ ]:
x=company.iloc[:,1:]
x

,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,138,73,11,276,120,0,42,17,1,1
1,111,48,16,260,83,1,65,10,1,1
2,113,35,10,269,80,2,59,12,1,1
3,117,100,4,466,97,2,55,14,1,1
4,141,64,3,340,128,0,38,13,1,0
...,...,...,...,...,...,...,...,...,...,...
395,138,108,17,203,128,1,33,14,1,1
396,139,23,3,37,120,2,55,11,0,1
397,162,26,12,368,159,2,40,18,1,1
398,100,79,7,284,95,0,50,12,1,1


In [ ]:
y=company.iloc[:,1]
y

0      138
1      111
2      113
3      117
4      141
      ... 
395    138
396    139
397    162
398    100
399    134
Name: CompPrice, Length: 400, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [ ]:
cart = DecisionTreeRegressor()
n_trees=10
model=BaggingRegressor(base_estimator=cart,n_estimators=n_trees,random_state=42)
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
y_pred

array([118.3, 131. , 139.6, 146. , 145. , 118.7,  94.7, 153.7, 137. ,
       110. , 132.8,  97.9, 112.1, 108. , 132.2, 122. , 133. , 140.8,
       107.1, 123.1, 137.1, 122. , 113. , 121. , 138. , 130. , 134. ,
       131.9, 143.5, 126.9, 123.1, 110.7, 139.1, 119.3, 135. , 150.2,
       137.1, 117.9, 165.6, 135. , 132.1, 123. , 126.2, 114.9, 126.9,
       114. , 141.5, 116. , 123.1, 125. , 131.1, 132.8, 110.9, 123.1,
       107.8, 122. , 107.9, 131. , 124. , 118.8, 118.9, 132.8, 112. ,
       130. , 149.7, 128.1, 122. , 125. , 114.9, 102.8, 102.9, 103.8,
       114.8, 126.2, 134. , 124. , 147.5, 116. , 124. , 111. , 122. ,
       117.2, 150.3, 122. , 162.5, 124. , 122. ,  97.8, 136. , 142.9,
       140.8, 123.1, 119.1, 109.1, 134. , 138. , 131.9,  99.9, 118.4,
        89.4])

In [ ]:
model.score(x_train,y_train)

0.9989594255666874

In [ ]:
model.score(x_test,y_test)

0.9971537011292118

In [ ]:
model_1=RandomForestRegressor(n_estimators=500,max_features=4,random_state=40)
model_1.fit(x_train,y_train)

RandomForestRegressor(max_features=4, n_estimators=500, random_state=40)

In [ ]:
y_pred=model_1.predict(x_test)

In [ ]:
model_1.score(x_train,y_train)

0.993461269348662

In [ ]:
model_1.score(x_test,y_test)

0.9702955742503278

In [ ]:
cvr=cross_val_score(model_1,x_train,y_train,cv=10)
cvr.mean()

0.9542742831673904

In [ ]:
cvr=cross_val_score(model_1,x_test,y_test,cv=10)
cvr.mean()

0.8968362381570106